In [ ]:
import json

experiment = '5dcbed98-12ef-48d4-81cd-07a14ee1d4e5'

with open(f'/Users/wakephul/fun/musin/backend/simulations/output/{experiment}/output/files/info.json') as f:
    data = json.load(f)

print(data)

In [2]:
import pdb
import numpy as np

def calculate_bins(senders, times, number_monitored_neurons, bin_size = 5, start_time = 5, end_time = 1000, split_into = 1):

    print("Calculating rates divided into bins")
    bins = list(range(int(start_time), int(end_time)+1, bin_size))

    monitored_times = {}

    for index, time in enumerate(times):
        bin_index = np.digitize(time, bins, right=True)
        if bin_index < len(bins):
            bin_time = np.take(bins, bin_index)
            if bin_time in monitored_times:
                monitored_times[bin_time].append(senders[index])
            else:
                monitored_times[bin_time] = [senders[index]]
    for id in monitored_times:
        monitored_times[id].sort()

    bin_rates = {}

    for bin_time in bins:
        if bin_time in monitored_times:
            bin_rate = len(monitored_times[bin_time]) * 1000 / (bin_size * number_monitored_neurons)
            bin_rates[bin_time] = bin_rate
        else:
            bin_rates[bin_time] = 0
        
    bin_rates_split = {i: [] for i in range(int(split_into))}

    step = int((end_time-start_time)/split_into)
    start = int(start_time)
    end = int(end_time)
    for index, value in enumerate(range(start, end, step)):
        bin_rates_split[index] = {k:v for k, v in bin_rates.items() if (k>=value and k<value+step)}

    return bin_rates_split

def calculate_response_times(values, threshold, trial_time, bin_size):
    elements_for_trial = trial_time/bin_size
    values_into_trials = divide_into_trials(values, elements_for_trial)
    response_times = []
    for values_index, trial_values in enumerate(values_into_trials):
        time_ids = [time_id for time_id, value in enumerate(trial_values) if value >= threshold]
        if len(time_ids):
            first_time_id = time_ids[0]
            actual_time = (values_index*trial_time)+(first_time_id*bin_size)
            response_times.append(actual_time)
        else:
            response_times.append(999999999)
    return response_times

def divide_into_trials(seq, size):
    return [seq[int(pos):(int(pos) + int(size))] for pos in range(0, len(seq), int(size))]

In [3]:
import matplotlib.pyplot as plt

def moving_average_plot_no_save(plot_data):

    times = list(map(int, list(plot_data.keys())))
    values = list(plot_data.values())
    values = [x for _, x in sorted(zip(times, values))]
    times = sorted(times)

    window_size = 5
    i = window_size//2
    ma = []

    values = [0 for x in range(window_size//2)] + values + [0 for x in range(window_size//2)]

    while i < (len(values) - window_size//2):
        
        window = values[(i - window_size//2) : (i + window_size//2)]
        window_average = round(sum(window) / window_size, 2)
        
        ma.append(window_average)
        
        i += 1

    return ma, times

def cdf_calc(data):
    count, bins_count = np.histogram(data, bins=10)
    try:
        pdf = count / sum(count)
    except:
        pdf = 0
    cdf = np.cumsum(pdf)
    return bins_count, cdf

In [ ]:
bin_size = 5

times_spike_monitor_DCN_a = [t for t in data['times_spike_monitor_DCN_a'] if t > data['train_time']]
times_spike_monitor_DCN_b = [t for t in data['times_spike_monitor_DCN_b'] if t > data['train_time']]

print(times_spike_monitor_DCN_a)

senders_spike_monitor_DCN_a = data['senders_spike_monitor_DCN_a']
senders_spike_monitor_DCN_b = data['senders_spike_monitor_DCN_b']
len_idx_monitored_neurons_DCN_a = data['len_idx_monitored_neurons_DCN_a']
len_idx_monitored_neurons_DCN_b = data['len_idx_monitored_neurons_DCN_b']
train_time = data['train_time']
test_time = data['test_time']
test_number = data['test_number']
test_types = data['test_types']

bin_rates_DCN_complete_a = calculate_bins(senders_spike_monitor_DCN_a, times_spike_monitor_DCN_a, (len_idx_monitored_neurons_DCN_a)//2, bin_size, train_time, train_time+(test_time*test_number), test_number)
bin_rates_DCN_complete_b = calculate_bins(senders_spike_monitor_DCN_b, times_spike_monitor_DCN_b, (len_idx_monitored_neurons_DCN_b)//2, bin_size, train_time, train_time+(test_time*test_number), test_number)

cdf_plots = []

In [ ]:
threshold = 10
for tt_index, tt in enumerate(test_types):
    bin_rates_a_portion = bin_rates_DCN_complete_a[tt_index]
    bin_rates_b_portion = bin_rates_DCN_complete_b[tt_index]

    ma_rates_a, times_a = moving_average_plot_no_save(bin_rates_a_portion)
    ma_rates_b, times_b = moving_average_plot_no_save(bin_rates_b_portion)

    # plt.plot(times_a, ma_rates_a, label='DCN A')
    # plt.show()
    # plt.plot(times_b, ma_rates_b, label='DCN B')
    # plt.show()

    ma_rates = [abs(a-b) for a, b in zip(ma_rates_a, ma_rates_b)]

    trial_time = 1000

    response_times = []
    resp = calculate_response_times(ma_rates, threshold, trial_time, bin_size)
    for rt in resp:
        response_times.append(rt%1000)

    print('RESPONSE TIMES: ', response_times)

    bins_count, cdf = cdf_calc(response_times)
    
    plt.xlim(0, 600)
    plt.ylim(0, 1.1)
    plt.plot(bins_count[1:], cdf, '--o', label="CDF cerebellum")
    plt.legend()
    plt.show()

In [ ]:
from graphviz import Digraph

# Define table metadata
tables = {
    "execution": [
        {"name": "code", "type": "String(36)", "primary_key": True},
        {"name": "name", "type": "String(100)", "primary_key": False},
        {"name": "created_at", "type": "DateTime", "primary_key": False},
        {"name": "finished_at", "type": "DateTime", "primary_key": False},
    ],
    "execution_network_side_input_relationship": [
        {"name": "execution_code", "type": "String(36)", "primary_key": True, "foreign_key": "execution.code"},
        {"name": "network_code", "type": "String(36)", "primary_key": True, "foreign_key": "network.code"},
        {"name": "side", "type": "Integer", "primary_key": True},
        {"name": "input_code", "type": "String(36)", "primary_key": True, "foreign_key": "input.code"},
    ],
    "execution_result": [
        {"name": "id", "type": "Integer", "primary_key": True},
        {"name": "result_path", "type": "String(100)", "primary_key": False},
        {"name": "image_path", "type": "String(100)", "primary_key": False},
    ],
    "input": [
        {"name": "code", "type": "String(36)", "primary_key": True},
        {"name": "name", "type": "String(255)", "primary_key": False},
        {"name": "multiple", "type": "Boolean", "primary_key": False},
        {"name": "rate_start", "type": "Float", "primary_key": False},
        {"name": "rate_end", "type": "Float", "primary_key": False},
        {"name": "rate_step", "type": "Float", "primary_key": False},
        {"name": "first_spike_latency_start", "type": "Float", "primary_key": False},
        {"name": "first_spike_latency_end", "type": "Float", "primary_key": False},
        {"name": "first_spike_latency_step", "type": "Float", "primary_key": False},
        {"name": "number_of_neurons_start", "type": "Integer", "primary_key": False},
        {"name": "number_of_neurons_end", "type": "Integer", "primary_key": False},
        {"name": "number_of_neurons_step", "type": "Integer", "primary_key": False},
        {"name": "trial_duration_start", "type": "Integer", "primary_key": False},
        {"name": "trial_duration_end", "type": "Integer", "primary_key": False},
        {"name": "trial_duration_step", "type": "Integer", "primary_key": False},
        {"name": "created_at", "type": "DateTime", "primary_key": False},
    ],
    "network": [
        {"name": "code", "type": "String(36)", "primary_key": True},
        {"name": "name", "type": "String(100)", "primary_key": False},
        {"name": "sides", "type": "Integer", "primary_key": False},
        {"name": "created_at", "type": "DateTime", "primary_key": False},
    ],
    "network_parameter": [
        {"name": "network_code", "type": "String(36)", "primary_key": True, "foreign_key": "network.code"},
        {"name": "name", "type": "String(100)", "primary_key": True},
        {"name": "value", "type": "String(100)", "primary_key": False},
    ],
}

# Generate class diagram
def create_class_diagram(tables, output_file="class_diagram"):
    dot = Digraph("Class Diagram", format="png")
    dot.attr(rankdir="LR", fontsize="10", ratio="compress")
    
    # Add nodes for tables
    for table_name, columns in tables.items():
        label = f"{table_name}|"
        for col in columns:
            pk = " (PK)" if col.get("primary_key", False) else ""
            label += f"{col['name']} : {col['type']}{pk}\\l"
        dot.node(table_name, label=f"{{ {label} }}", shape="record")
    
    # Add edges for relationships
    for table_name, columns in tables.items():
        for col in columns:
            if "foreign_key" in col:
                target_table = col["foreign_key"].split(".")[0]
                dot.edge(table_name, target_table, arrowhead="crow", label="FK")

    dot.render(output_file, cleanup=True)
    print(f"Class diagram saved as {output_file}.png")

# Run the function
create_class_diagram(tables)